# Interview Coach - Демонстрация

In [35]:
import sys
sys.path.insert(0, '..')

## 1. Модели данных

In [36]:
from interview_coach.models.candidate import Candidate
from interview_coach.models.feedback import (
    FinalFeedback, Decision, Grade, HiringRecommendation, 
    SkillAssessment, KnowledgeGap
)

candidate = Candidate(
    name="Иван Иванов",
    position="Backend Developer",
    expected_grade="Junior",
    experience="1 год работы с Python и Django"
)

print("Контекст кандидата:")
print(candidate.get_context_string())

Контекст кандидата:
Кандидат: Иван Иванов
Позиция: Backend Developer
Ожидаемый грейд: Junior
Опыт: 1 год работы с Python и Django


In [37]:
decision = Decision(
    grade=Grade.JUNIOR,
    hiring_recommendation=HiringRecommendation.HIRE,
    confidence_score=75
)

skills = [
    SkillAssessment(topic="Python", evidence="Хорошо объяснил GIL", score=7),
    SkillAssessment(topic="Django", evidence="Знает ORM", score=6),
]

gaps = [
    KnowledgeGap(topic="Docker", user_answer="Не знаю", correct_answer="Контейнеризация приложений"),
]

feedback = FinalFeedback(
    decision=decision,
    confirmed_skills=skills,
    knowledge_gaps=gaps,
    summary="Хороший Junior с потенциалом роста"
)

print(feedback.to_formatted_string())


[==SEPARATE==]

           ФИНАЛЬНЫЙ ФИДБЭК ИНТЕРВЬЮ

[==SEPARATE==]


ВЕРДИКТ
---[[[SubSep]]]---
  Уровень: Junior
  Рекомендация: Hire
  Уверенность: 75%

ТЕХНИЧЕСКИЕ НАВЫКИ
---[[[SubSep]]]---
  Подтвержденные навыки:
            Python: Хорошо объяснил GIL
            Django: Знает ORM

  Пробелы в знаниях:
            Docker
                      Ваш ответ: Не знаю
                      Правильный ответ: Контейнеризация приложений

РЕЗЮМЕ
---[[[SubSep]]]---
  Хороший Junior с потенциалом роста


[==SEPARATE==]



## 2. Пресеты личности кандидата

In [38]:
from interview_coach.agents.candidate import CandidateTraits, CandidatePresets

presets = {
    "strong_junior": CandidatePresets.strong_junior(),
    "weak_junior": CandidatePresets.weak_junior(),
    "confident_wrong": CandidatePresets.confident_wrong(),
    "nervous": CandidatePresets.nervous(),
    "experienced": CandidatePresets.experienced(),
}

for name, traits in presets.items():
    print(f"{name}: знания={traits.knowledge_level:.1f}, уверенность={traits.confidence:.1f}")

strong_junior: знания=0.6, уверенность=0.6
weak_junior: знания=0.3, уверенность=0.3
confident_wrong: знания=0.4, уверенность=0.9
nervous: знания=0.5, уверенность=0.3
experienced: знания=0.8, уверенность=0.7


In [39]:
custom_traits = CandidateTraits(
    knowledge_level=0.7,
    confidence=0.4,
    honesty=0.9,
    verbosity=0.6,
    off_topic_tendency=0.1,
    nervousness=0.5,
    curiosity=0.8
)

print("Описание личности:")
print(custom_traits.generate_personality_description())

Описание личности:
Ты - кандидат со следующими чертами личности:

УРОВЕНЬ ЗНАНИЙ: Хорошие знания, решает задачи
УРОВЕНЬ ЧЕСТНОСТИ: Максимально честен.

МАНЕРА ОБЩЕНИЯ: Подробные объяснения
УРОВЕНЬ БОЛТЛИОВОСТИ: Строго по теме. Постоянно задаёт вопросы.

ЭМОЦИОНАЛЬНОЕ СОСТОЯНИЕ: Заметно нервничает
УРОВЕНЬ УВЕРЕННОСТИ: Умеренная уверенность.

ПОВЕДЕНИЕ: На основе своих черт веди себя естественно.


## 3. Промпты

In [40]:
from interview_coach.utils.prompts import Prompts

observer_prompt = Prompts.get_observer_prompt(
    candidate_info="Иван, Backend Developer, Junior",
    conversation_history=["Интервьюер: Что такое GIL?", "Кандидат: Global Interpreter Lock"],
    current_message="Это механизм синхронизации в CPython",
    topics_covered=["Python basics"]
)

print("Observer prompt:")
print(observer_prompt[:500] + "...")

Observer prompt:
Проанализируй последний ответ кандидата.

[==SEPARATE==]

ИНФОРМАЦИЯ О КАНДИДАТЕ:
Иван, Backend Developer, Junior

[==SEPARATE==]

ИСТОРИЯ ДИАЛОГА:
Интервьюер: Что такое GIL?
Кандидат: Global Interpreter Lock

[==SEPARATE==]

ПОСЛЕДНЕЕ СООБЩЕНИЕ КАНДИДАТА:
Это механизм синхронизации в CPython

[==SEPARATE==]

ТЕМЫ, КОТОРЫЕ УЖЕ ОБСУЖДАЛИСЬ:
Python basics

[==SEPARATE==]

Дай анализ в JSON формате....


## 4. Память диалога

In [41]:
from interview_coach.core.memory import ConversationMemory
from interview_coach.models.turn import Turn, ObserverAnalysis, ResponseQuality, ConfidenceLevel

memory = ConversationMemory()

for i in range(3):
    analysis = ObserverAnalysis(
        quality_score=6 + i,
        quality_level=ResponseQuality.GOOD,
        confidence=ConfidenceLevel.MEDIUM,
        topics_mentioned=[f"Topic_{i}"],
        recommendation="Продолжать"
    )
    turn = Turn(
        turn_id=i+1,
        agent_visible_message=f"Вопрос {i+1}",
        user_message=f"Ответ {i+1}",
        observer_analysis=analysis
    )
    memory.add_turn(turn)

print(f"Количество ходов: {memory.get_turn_count()}")
print(f"Средний скор: {memory.get_average_score():.1f}")
print(f"Темы: {memory.topics_covered}")

Количество ходов: 3
Средний скор: 7.0
Темы: ['Topic_0', 'Topic_1', 'Topic_2']


## 5. Анализ ответов

In [42]:
from interview_coach.models.turn import ObserverAnalysis, InterviewerInstruction, ResponseQuality, ConfidenceLevel

analysis = ObserverAnalysis(
    quality_score=8,
    quality_level=ResponseQuality.GOOD,
    confidence=ConfidenceLevel.HIGH,
    hallucination_detected=True,
    hallucination_details="Python 3 вышел в 2008, не 2010",
    correct_points=["Синтаксис", "ООП"],
    incorrect_points=["Дата релиза"],
    recommendation="Уточнить даты"
)

print("Внутренние мысли Observer:")
print(analysis.to_internal_thoughts())

Внутренние мысли Observer:
Оценка: 8/10 (good). [HALLUCINATION] Python 3 вышел в 2008, не 2010. Верно: Синтаксис, ООП. Ошибки: Дата релиза. Рекомендация: Уточнить даты.


In [43]:
instruction = InterviewerInstruction(
    action="correct",
    difficulty_level=3,
    correction_needed=True,
    correction_text="Python 3 был выпущен в декабре 2008 года"
)

print("Внутренние мысли Interviewer:")
print(instruction.to_internal_thoughts())

Внутренние мысли Interviewer:
Поправлю (сложность 3/5). Исправление: 'Python 3 был выпущен в декабре 2008 года...'.


## 6. Конфигурация

In [44]:
from interview_coach.config import config

print(f"Provider: {config.llm.provider}")
print(f"Local model: {config.llm.local_model}")
print(f"Local URL: {config.llm.local_model_url}")

Provider: LLMProvider.LOCAL
Local model: qwen2.5:7b
Local URL: http://localhost:11434/v1


## 7. Запуск автоматического интервью

In [45]:
from interview_coach.core.auto_orchestrator import AutoInterviewOrchestrator, AutoInterviewConfig
from interview_coach.models.candidate import Candidate
from interview_coach.llm.huggingface_provider import LocalLLMProvider

llm = LocalLLMProvider(base_url="http://localhost:11434", model="mistral-nemo")

candidate = Candidate(
    name="Иван Иванов",
    position="Backend Developer",
    expected_grade="Junior",
    experience="1 год Python и Django"
)

auto_config = AutoInterviewConfig(
    max_turns=5,
    candidate_personality="strong_junior"
)

orchestrator = AutoInterviewOrchestrator(candidate=candidate, llm=llm, auto_config=auto_config)
feedback = orchestrator.run_full_interview()

print(feedback.to_formatted_string())


[==SEPARATE==]

           ФИНАЛЬНЫЙ ФИДБЭК ИНТЕРВЬЮ

[==SEPARATE==]


ВЕРДИКТ
---[[[SubSep]]]---
  Уровень: Junior
  Рекомендация: Hire
  Уверенность: 85%

ТЕХНИЧЕСКИЕ НАВЫКИ
---[[[SubSep]]]---
  Подтвержденные навыки:
            Python programming: Used Python in both projects.
            Django framework: Used Django in both projects.
            Django Celery for background tasks: Used it in task management project.
            Django Email Sending: Used it in task management project.
            Pytest for testing Django projects: Mentioned using pytest in the interview.

  Пробелы в знаниях:
            Django REST Framework
                      Ваш ответ: Used it in food ordering project but didn't mention details.
                      Правильный ответ: Should have mentioned how he used authentication, serialization and viewsets.
            PostgreSQL database management
                      Ваш ответ: Mentioned using PostgreSQL but didn't provide details.
               